# Query models

Python API to define query models

A [QueryModel](reference-api.rst#vespa.query.QueryModel) is an abstraction
that encapsulates all the relevant information controlling how your app match and rank documents.
A `QueryModel` can be used for [querying](reference-api.rst#vespa.application.Vespa.query),
[evaluating](reference-api.rst#vespa.application.Vespa.evaluate)
and [collecting data](reference-api.rst#vespa.application.Vespa.collect_training_data) from an app.

Before version `0.5.0`, the only way to build a `QueryModel` was by specifying arguments like `match_phase`
and `rank_profile` using the pyvespa API,
such as _match operators_ like [OR](reference-api.rst#vespa.query.OR) - e.g.:

In [1]:
from learntorank.query import QueryModel, Ranking, OR

standard_query_model = QueryModel(
    name="or_bm25",
    match_phase = OR(),
    ranking = Ranking(name="bm25")
)

Starting in version `0.5.0` we can bypass the pyvespa high-level API and create a `QueryModel` with the full flexibility of the [Vespa Query API](https://docs.vespa.ai/en/reference/query-api-reference.html). This is useful for use cases not covered by the pyvespa API and for users that are familiar with and prefer to work with the Vespa Query API.

In [2]:
def body_function(query):
    body = {'yql': 'select * from sources * where userQuery();',
            'query': query,
            'type': 'any',
            'ranking': {'profile': 'bm25', 'listFeatures': 'false'}}
    return body

flexible_query_model = QueryModel(body_function = body_function)

The `flexible_query_model` defined above is equivalent to the `standard_query_model`, as we can see when querying the `app`. We will use the [cord19 app](https://cord19.vespa.ai/) in our demonstration.

In [3]:
from vespa.application import Vespa

app = Vespa(url = "https://api.cord19.vespa.ai")

In [4]:
from learntorank.query import send_query

standard_result = send_query(
    app=app, 
    query="this is a test", 
    query_model=standard_query_model
)
standard_result.get_hits().head(3)

,qid,doc_id,score,rank
0,0,id:covid-19:doc::142863,11.824458,0
1,0,id:covid-19:doc::187156,11.818079,1
2,0,id:covid-19:doc::31328,11.288179,2


In [5]:
flexible_result = send_query(
    app=app, 
    query="this is a test", 
    query_model=flexible_query_model
)
flexible_result.get_hits().head(3)

,qid,doc_id,score,rank
0,0,id:covid-19:doc::142863,11.824458,0
1,0,id:covid-19:doc::187156,11.818079,1
2,0,id:covid-19:doc::31328,11.288179,2
